In [1]:
from twnews.utils.memoize import load

In [2]:
news, tweets =load('dataset_applied')

In [3]:
print tweets[:10]
url_to_news = {}
for idx, _news in enumerate(news):
    url_to_news[_news.link] = (idx, _news)
    

[TWEET: Эрдогана пригласили в качестве свидетеля на еврейскую свадьбу: Президент Турции Реджеп Эрдоган будет присутств... , TWEET: Бессонница оказалась связана с нарушением структуры мозга - Исследователи из 2-о1 Народной больницы провинции Г... , TWEET: Бессонница оказалась связана с нарушением структуры мозга  #TeamFollowBack, TWEET: Эквадор подтвердил встречу нефтедобывающих стран Латинской Америки  © AP Photo , TWEET: Эрдогана пригласили в качестве свидетеля на еврейскую свадьбу , TWEET: МИД РФ: Афганистан может стать плацдармом для экспансии ИГ  #RUS, TWEET: Кадыров заявил о готовности чеченских силовиков служить в Нацгвардии: Глава Чечни Рамзан Кадыров поддержал реш... , TWEET: Приближенные наркобарона Коротышки фигурируют в "панамских бумагах" , TWEET: Правительство обсудит изменения в налоговую конвенцию с Австрией: Кабинет министров в среду рассмотрит проект ... , TWEET: Начальник отдела призыва военкомата Москвы арестован по делу о взятках: Начальник отдела подготовки и призы

In [4]:
import logging
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import logging
from scipy import sparse
from twnews.utils.extra import progressbar_iterate



def convert_to_compare_matrix(documents):
    dim = documents[0].compare_vector.shape[0]

    data, row_idxs, column_idxs = [], [], []
    for column_idx, document in enumerate(documents):
        rows, _, values = sparse.find(document.compare_vector)
        for i, value in enumerate(values):
            data.append(values[i])
            row_idxs.append(rows[i])
            column_idxs.append(column_idx)

    compare_matrix = sparse.csr_matrix((data, (row_idxs, column_idxs)), shape=(dim, len(documents)))
    return compare_matrix

logging.info('convert tweets to compare matrix')
tweets_matrix = convert_to_compare_matrix(tweets)

logging.info('convert news to compare matrix')
news_matrix = convert_to_compare_matrix(news)

logging.info('build cosine similarity matrix')
mat = cosine_similarity(tweets_matrix.T, news_matrix.T)


# for tweet in tweets:
#     print tweet
#     for url in tweet.urls:
#         print url_to_news[url]
        


In [6]:
res = []
for tweet_idx, tweet in enumerate(tweets):
    for url in tweet.urls:
        if url in url_to_news:
            news_idx, _news = url_to_news[url]
            score = mat[tweet_idx][news_idx]
            res.append((score, tweet, _news))
            break
        

In [9]:
sorted(res, key=lambda x: x[0])[:1000]

[(0.0,
  TWEET: Узнаю почерк Влада Путина ,
  NEWS: В Японии пропал с радаров самолет Сил самообороны),
 (0.0,
  TWEET: только такие дебилы и могут стрелки переводить ,
  NEWS: Нацгвардия РФ будет при необходимости применять боевую технику для пресечения массовых беспорядков),
 (0.0,
  TWEET: какие обидчивые... ,
  NEWS: Партия Порошенко потребует от NYT опровержения статьи о «коррумпированном болоте» на Украине),
 (0.0,
  TWEET: Шапито в голодной Европе ,
  NEWS: Акционеры Daimler разругались из-за сосисок),
 (0.0,
  TWEET: прежде чем нести БРЕД с начало в инете посмотри если не знаешь ,
  NEWS: Роскосмос: Поставка российских ракетных двигателей Китаю откладывается),
 (0.0,
  TWEET: Я о том, что не нужно лезть в распоряжение финансами, если Вы ничего не понимаете в управлениями финансами. ,
  NEWS: Мария Захарова раскрыла комичную деталь голландского референдума),
 (0.0,
  TWEET: Уорхохол  #Politcreative ,
  NEWS: Украинец Энди Уорхол: в Закарпатье появится площадь в честь «соотечеств